#### Note:
**2008.csv** file is too large so it is not in my git repository. If you want to run the scripts then you may have to download the file from http://stat-computing.org/dataexpo/2009/the-data.html.

*** Changes in Second version ***

This is the second version of parsing data jupyter file. The edges data now include the number of flight during the specified month.

# Parsing the data
### Final Project - Data Visualization Course
### Udacity Data Analyst Nanodegree

For this final project I will use 2008 flights data that I downloaded from http://stat-computing.org/dataexpo/2009/the-data.html. The file doesn't have the geographic coordination of each airport so I downloaded the file that contains the information of coordinates of each airport from https://gist.githubusercontent.com/tdreyno/4278655/raw/7b0762c09b519f40397e4c3e100b097d861f5588/airports.json.


First I will load the json file and save it as a dictionary.

In [4]:
### import necessary libraries
import json

### Create an empty dictionary
airport_dict = {}

### load the json file
count = 0
with open("airport.json") as F:
    for i in json.load(F):
        try:
            airport_dict[i['code']] = {'lon': i['lon'], 'lat': i['lat']}
        except:
            print i['code'], ":", i['name']
            count += 1

In [5]:
print "Saved airports:",len(airport_dict)
print "Excluded airports:", count

Saved airports: 3885
Excluded airports: 0


Now we will load the flights file and parse the data.

Since there are too many records in this file I will filter the **February** data only. Only the records containing **OGD** are excluded because **airport_dict** dictionary doesn't contain this coordinate information. Everything else is saved as a csvfile.

In [6]:
### load csv library
import csv

### Dictionary for avoiding redundancy
redun = {}

### load csv data
with open("2008.csv") as csvfile:
    for i in csv.DictReader(csvfile):
        if int(i["Month"]) == 2:
            if i['Dest'] != 'OGD':
                ### Define the key avoiding redundancy.
                if i['Dest'] < i['Origin']:
                    DO = i['Dest']+"-"+i['Origin']
                    Dest = 0
                else:
                    DO = i['Origin']+"-"+i['Dest']
                    Dest = 1
                
                ### Save the data in redun dictionary
                if DO not in redun.keys():
                    if Dest == 0:
                        redun[DO] = {"flights": 1, 
                                     "Dest_lon": airport_dict[i['Dest']]['lon'],
                                     "Dest_lat": airport_dict[i['Dest']]['lat'],
                                     "Origin_lon": airport_dict[i['Origin']]['lon'],
                                     "Origin_lat": airport_dict[i['Origin']]['lat']}
                    else:
                        redun[DO] = {"flights": 1, 
                                     "Dest_lon": airport_dict[i['Origin']]['lon'],
                                     "Dest_lat": airport_dict[i['Origin']]['lat'],
                                     "Origin_lon": airport_dict[i['Dest']]['lon'],
                                     "Origin_lat": airport_dict[i['Dest']]['lat']}
                else:
                    redun[DO]["flights"] += 1

Now the flight data of February is saved as **redun**. Let's look at the first 10 items of the dictionary.

In [7]:
num = 1
for k,v in redun.iteritems():
    print "{}\n\tDest lat: {}, Dest lon: {}".format(k, v['Dest_lat'], v['Dest_lon'])
    print "\tOrigin lat: {}, Origin lon: {}".format(v['Origin_lat'], v['Origin_lon'])
    if num == 10:
        break
    num+=1

IAD-SFO
	Dest lat: 38.9556, Dest lon: -77.4484
	Origin lat: 37.6148, Origin lon: -122.392
HNL-SEA
	Dest lat: 21.3358, Dest lon: -157.919
	Origin lat: 47.4405, Origin lon: -122.296
OKC-STL
	Dest lat: 35.3981, Dest lon: -97.5974
	Origin lat: 38.7414, Origin lon: -90.3647
MEM-MLI
	Dest lat: 35.047, Dest lon: -89.9823
	Origin lat: 41.4539, Origin lon: -90.5058
MDW-RSW
	Dest lat: 41.7875, Dest lon: -87.7416
	Origin lat: 26.5228, Origin lon: -81.7531
DFW-MOB
	Dest lat: 32.9222, Dest lon: -97.0409
	Origin lat: 30.6775, Origin lon: -88.243
BOI-LAS
	Dest lat: 43.5709, Dest lon: -116.222
	Origin lat: 36.0806, Origin lon: -115.143
COS-SAN
	Dest lat: 38.7829, Dest lon: -104.697
	Origin lat: 32.7299, Origin lon: -117.195
BNA-BWI
	Dest lat: 36.1342, Dest lon: -86.6682
	Origin lat: 39.1841, Origin lon: -76.6745
BOI-LAX
	Dest lat: 43.5709, Dest lon: -116.222
	Origin lat: 33.9456, Origin lon: -118.391


Let's look at the number of items in the dictionary.

In [8]:
len(redun)

2339

There are **2339** items in the dictionary. In other words, there are that much of routes that can be visualized on the map. However, too many nodes and edges only confuse readers.

First, I will divide the **key** into **Destination** and **Origin** separately and save each airport individually.

In [46]:
### iterate over redun dicationary
DO_flights = {}
for k,v in redun.iteritems():
    airport_A = k.split("-")[0]
    airport_B = k.split("-")[1]
    if airport_A in DO_flights.keys():
        DO_flights[airport_A] += v['flights']
    else:
        DO_flights[airport_A] = v['flights']
        
    if airport_B in DO_flights.keys():
        DO_flights[airport_B] += v['flights']
    else:
        DO_flights[airport_B] = v['flights']

len(DO_flights)

285

In [47]:
### Sort by value
import pprint
import operator

Dest_sorted = sorted(DO_flights.items(), key=operator.itemgetter(1))

### Print out the result
print "Destinations and flights"
top_Dest = ()
for i in range(1,31):
    print Dest_sorted[-i*2]
    top_Dest += (Dest_sorted[-i*2][0],)

Destinations and flights
('ORD', 55959)
('DEN', 37327)
('PHX', 33197)
('LAS', 28562)
('SLC', 23368)
('MSP', 21701)
('MCO', 21406)
('LGA', 19491)
('BOS', 18856)
('SEA', 15955)
('PHL', 15618)
('MDW', 14529)
('MEM', 13297)
('IAD', 12822)
('OAK', 10896)
('MIA', 10602)
('CLE', 10165)
('RDU', 9763)
('BNA', 9115)
('HOU', 8887)
('DAL', 8723)
('SNA', 8001)
('PIT', 7123)
('MSY', 6634)
('ABQ', 6358)
('RSW', 5621)
('PBI', 5459)
('BUR', 5195)
('TUS', 4787)
('OKC', 4264)


Above is the list of **30** destinations that will be used in the data visualization project. Let's filter the **redun** dictionary with these detinations. 

In [48]:
### Filtering redun dictionary
redun_filtered = {}

for k,v in redun.iteritems():
    if k.split("-")[0] in top_Dest and k.split("-")[1] in top_Dest:
        redun_filtered[k] = v

len(redun_filtered)

258

There are total **30** nodes **96** edges. 


In this data visualization project, nodes and links properties in **D3.js** will be used to indicate the flight paths. So, I will parse the data to create arrays for nodes and links. Let's count the number of edges of each node and save it as **nodes.csv** file.

In [49]:
### Create arrays of the number of edges of each node
edges_count = {}
for dest in top_Dest:
    edges_count[dest] = 0
    for k in redun_filtered.keys():
        if dest in k.split("-"):
            edges_count[dest] += 1

edges_count

{'ABQ': 15,
 'BNA': 19,
 'BOS': 20,
 'BUR': 6,
 'CLE': 21,
 'DAL': 4,
 'DEN': 28,
 'HOU': 12,
 'IAD': 22,
 'LAS': 26,
 'LGA': 16,
 'MCO': 22,
 'MDW': 20,
 'MEM': 17,
 'MIA': 19,
 'MSP': 23,
 'MSY': 18,
 'OAK': 14,
 'OKC': 12,
 'ORD': 24,
 'PBI': 7,
 'PHL': 21,
 'PHX': 25,
 'PIT': 15,
 'RDU': 17,
 'RSW': 11,
 'SEA': 20,
 'SLC': 23,
 'SNA': 8,
 'TUS': 11}

In [50]:
### Save the tuple as a csv file
ind = 0
nodes = {}
with open("nodes.csv", "wb") as F:
    fieldnames = ['code', 'id', 'size', 'lon', 'lat'] 
    writer = csv.DictWriter(F, fieldnames=fieldnames)
    writer.writeheader()

    ### write rows
    for dest in top_Dest:
        tmp_dict = {}
        tmp_dict['code'] = dest
        tmp_dict['id'] = ind
        tmp_dict['lon'] = airport_dict[dest]['lon']
        tmp_dict['lat'] = airport_dict[dest]['lat']
        tmp_dict['size'] = edges_count[dest]
        
        nodes[dest] = ind
        writer.writerow(tmp_dict)
        ind += 1

Let's now save edges data.

In [51]:
### Save the edges as a csv file
with open("edges.csv", "wb") as F:
    fieldnames = ["source", "target", "Dest", "Origin", "flights"]
    writer = csv.DictWriter(F, fieldnames = fieldnames)
    writer.writeheader()
    
    ### iterate redun_filtered data and save edges
    for k,v in redun_filtered.iteritems():
        tmp_dict = {}
        tmp_dict["source"] = nodes[k.split("-")[0]]
        tmp_dict["target"] = nodes[k.split("-")[1]]
        tmp_dict["Dest"] = k.split("-")[0]
        tmp_dict["Origin"] = k.split("-")[1]
        tmp_dict["flights"] = v['flights']
        writer.writerow(tmp_dict)

Now the files are ready to be used in **D3.js** for data visualization. You can see the result in **final_project.html**.